In [23]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import requests
import json
import random
baseurl = "http://localhost:3000/"

In [25]:
users = []
toks = {}
with open("users.json") as u:
    users = json.loads(u.read())
for u in users:
    payload = {'username':u['username'],'password':u['password'],'appcode':u['appcode']}
    res = requests.post(f"{baseurl}auth/login/",json=payload)
    _t = json.loads(res.text)
    toks[u['username']] = _t["token"]

In [26]:
for u in users:
    headers = {
        "authorization":f"bearer {toks[u['username']]}"
    }
    print(u["valid_lp"][0], u["username"])
    payload = {
    "app_id": u["appcode"],
    "loyalty_pid": u["valid_lp"][0],
    "user_id": f"{u['username']}_id",
    "member_id": f"{u['username']}_m_id",
    "member_first": f"{u['member_first']}",
    "member_last":f"{u['member_last']}",
    "transaction_date": "20240808",
    "ref_num": f"{random.randint(100000,999999)}",
    "amount": 10000,
    "additional_info": "tonnes of additionl info"
    }
    res = requests.post(f"{baseurl}transact/add_record",json=payload,headers = headers)
    _t = json.loads(res.text)
    if(res.status_code!=200):
        print(f"{u['username']} payload transact fail - {res.status_code}")

BEST_MOVERS user1
user1 payload transact fail - 500
FRESH_BAG user2
user2 payload transact fail - 500
HIS_AND_HERS user3
user3 payload transact fail - 500
PASSION_PAY user4
user4 payload transact fail - 500
TAKO_RAMEN user5
user5 payload transact fail - 500
FRESH_BAG user6
user6 payload transact fail - 500
JOYSPRING_TOYS user7
user7 payload transact fail - 500
FRESH_BAG user8
user8 payload transact fail - 500
GO_CART user9
user9 payload transact fail - 500
KINGSMAN_SERVICES user10
user10 payload transact fail - 500
ROYAL_AIR user11
user11 payload transact fail - 500
OUTDOOR_BROS user12
user12 payload transact fail - 500
THE_NATURE_TREE user13
user13 payload transact fail - 500
BEST_MOVERS user14
user14 payload transact fail - 500
FRESH_BAG user15
user15 payload transact fail - 500
GOLDEN_HOTELS user16
user16 payload transact fail - 500
PASSION_PAY user17
user17 payload transact fail - 500
ROYAL_AIR user18
user18 payload transact fail - 500
SUSHI_CHOPSTICKS user19
user19 payload transac

In [27]:
import glob
listing = glob.glob('../AccrualFiles/*')
for l in listing:
    hbs = "transfer_date,amount,reference_number,outcome_code\n" 
    with open(l) as f:
        for ll in (f.read().split('\n'))[1::]:
            r = ll.split(',')
            if(len(r)>1):
                hbs+=f"{r[4]},{r[5]},{r[6]},0000\n"

    with open(f"../HandBackFiles/{l[16::].replace('ACCRUAL','HANDBACK')}",'w+') as ff:
        ff.write(hbs)
